In [7]:
from socceraction.data.wyscout import PublicWyscoutLoader
from socceraction.spadl.wyscout import convert_to_actions
from socceraction.data.opta import OptaLoader
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import math
import pickle
from mplsoccer import Pitch
import hashlib
import copy
import random
from os import system

In [8]:
# IMPORTANT CONSTANTS FOR THIS CODE SNIPPET
COLUMNS_XPASS_MODEL = ["path_id", "player_id_src", "player_id_target", "timestamp_origin", "timestamp_target", "start_x", "start_y", 
                       "end_x", "end_y", "distance_pass", "distance_sideline", "distance_goal", 
                       "distance_receiver_sideline", "distance_receiver_goal", "angle_pass", "distance_opponent",
                       "num_opponent_closer_goal", "distance_receiver_opponent", "num_opponent_closer_goal_receiver",
                       "num_opponent_in_path", "num_opponent_in_path_receiver"]
COLUMNS_XGOAL_MODEL = ["start_x", "start_y", "path_id", "timestamp", "player_id", "distance_to_goal", "angle_to_goal",
                       "distance_opponent", "num_opponent_closer_goal", "num_opponent_in_path"]
COLUMNS_PLAYERS_SKILL = ["acceleration", "aggression", "agility", "balance", "ball_control",
                        "composure", "crossing", "curve", "dribbling", "finishing",
                        "freekick_accuracy", "heading_accuracy", "interceptions", "jumping", "long_passing",
                        "long_shots", "marking", "penalties", "positioning", "reactions",
                        "shot_power", "sliding_tackle", "sprint_speed", "stamina", "short_passing",
                        "standing_tackle", "strength", "vision", "volleys"]
COLUMNS_XDRIBBLE_MODEL = ["path_id", "player_id", "timestamp_origin", "timestamp_target", "start_x", "start_y", "end_x", "end_y", 
                          "is_take_on", "distance_opponent", "num_opponent_closer_goal", "num_opponent_in_path", "distance_dribble"]
COLUMNS_PLAYERS_ATTRIBUTE = ["height_cm", "weight_kgs", "age"]

STANDARD_LENGTH_COURT = 105
STANDARD_WIDTH_COURT = 68
STANDARD_GOALLINE_WIDTH = 7.32
STANDARD_LENGTH_COURT_STATSBOMB = 120
STANDARD_WIDTH_COURT_STATSBOMB = 80
IS_HOME_TEAM_DIRECTION = True

INPUT_RAW_PLAYER_COORDINATE_FILEPATH = "data/input_decision_making_model/input_coordinate_player.csv"
INPUT_PLAYER_SKILL_FILE_CSV_PATH = 'data/input_decision_making_model/input_skill_player.csv'
INPUT_PLAYER_ATTRIBUTE_FILE_CSV_PATH = 'data/input_decision_making_model/input_attribute_player.csv'

In [9]:
# ALL HELPER FUNCTIONS
# Load Model from External Directory
def load_model_from_external_file_pickle(filename_path):
    return pickle.load(open(filename_path, 'rb'))

# Load Pandas Dataframe from external csv file
def load_pandas_dataframe_from_csv(filename_path, delimiter=","):
    return pd.read_csv(filename_path, delimiter=delimiter)

# Xpass : Helper to calculate nearest distance from sideline given coordinate
def calculate_nearest_distance_from_sideline(coordinate_x, coordinate_y):
    distance_to_left = coordinate_x
    distance_to_bottom = coordinate_y
    distance_to_top = STANDARD_WIDTH_COURT - coordinate_y
    distance_to_right = STANDARD_LENGTH_COURT - coordinate_x
    return min(distance_to_top, distance_to_bottom, distance_to_left, distance_to_right)

# Xpass : Helper to calculate distance with opposing goal given coordinate
def calculate_distance_with_opposing_goal(coordinate_x, coordinate_y, is_home_team):
    distance_vertical = abs((STANDARD_WIDTH_COURT / 2) - coordinate_y)
    if (is_home_team):
        distance_horizontal = STANDARD_LENGTH_COURT - coordinate_x
    else:
        distance_horizontal = coordinate_x
    return math.sqrt(distance_vertical ** 2 + distance_horizontal ** 2)

# Xpass or Xdribble : Helper to calculate distance between two coordinates (x1,y1) and (x2,y2)
def calculate_distance_between_two_coordinates(x1, y1, x2, y2):
    return math.sqrt(abs(x2-x1) ** 2 + abs(y2-y1) ** 2)

# Xpass : Helper to calculate angle between passer and receiver
def calculate_angle_pass(coordinate_x, coordinate_y, end_x, end_y, is_home_team):
    if (is_home_team):
        coordinate_x_goal = STANDARD_LENGTH_COURT
    else:
        coordinate_x_goal = 0
    coordinate_y_goal = STANDARD_WIDTH_COURT / 2

    distance_passer_to_goal = calculate_distance_between_two_coordinates(coordinate_x_goal, coordinate_y_goal, coordinate_x, coordinate_y)
    distance_receiver_to_goal = calculate_distance_between_two_coordinates(coordinate_x_goal, coordinate_y_goal, end_x, end_y)
    distance_passer_to_receiver = calculate_distance_between_two_coordinates(coordinate_x, coordinate_y, end_x, end_y)

    if (coordinate_x == end_x) and (coordinate_y == end_y):
        return 0
    else:
        cosine_value = (distance_passer_to_goal ** 2 + distance_passer_to_receiver ** 2 - distance_receiver_to_goal ** 2) / (2 * distance_passer_to_goal * distance_passer_to_receiver)
        if (cosine_value > 1):
            cosine_value = 1
        elif (cosine_value < -1):
            cosine_value = -1
        return math.acos(cosine_value)
    
# Opponent Feature : Calculate nearest opponent distance
def calculate_distance_opponent(start_x, start_y, list_node_opponents):
    list_distance_opponent = []
    for opponent_obj in list_node_opponents:
        opponent_x, opponent_y = opponent_obj.get_coordinate_value()
        distance_opponent = calculate_distance_between_two_coordinates(start_x, start_y, opponent_x, opponent_y)
        list_distance_opponent.append(distance_opponent)
    return min(list_distance_opponent) if len(list_distance_opponent) > 0 else 0

# Opponent Feature : Calculate number of opponents closer to goal
def calculate_num_opponent_closer_goal(start_x, start_y, list_node_opponents, is_home_team):
    if (is_home_team):
        coordinate_x_goal = STANDARD_LENGTH_COURT
    else:
        coordinate_x_goal = 0
    coordinate_y_goal = STANDARD_WIDTH_COURT / 2

    num_opponent_closer_to_goal = 0
    for opponent_obj in list_node_opponents:
        opponent_x, opponent_y = opponent_obj.get_coordinate_value()
        distance_passer_to_goal = calculate_distance_between_two_coordinates(start_x, start_y, coordinate_x_goal, coordinate_y_goal)
        distance_opponent_to_goal = calculate_distance_between_two_coordinates(opponent_x, opponent_y, coordinate_x_goal, coordinate_y_goal)
        if (distance_opponent_to_goal < distance_passer_to_goal):
            num_opponent_closer_to_goal += 1
    return num_opponent_closer_to_goal

# Opponent Feature : calculate number of opponents in path distance (default 10 metres)
def calculate_num_opponent_in_path(start_x, start_y, list_node_opponents):
    path_distance = 10
    num_opponent_in_path = 0
    for opponent_obj in list_node_opponents:
        opponent_x, opponent_y = opponent_obj.get_coordinate_value()
        distance_with_opponent = calculate_distance_between_two_coordinates(start_x, start_y, opponent_x, opponent_y)
        if (distance_with_opponent <= path_distance):
            num_opponent_in_path += 1
    return num_opponent_in_path

# Xgoal : Helper to calculate distance to opponent's goal
def calculate_distance_to_goal(length_court, width_court, coordinate_x, coordinate_y, is_home_team):
    if is_home_team:
        distance_to_goal = math.sqrt((abs(length_court - coordinate_x)) ** 2 + (abs((0.5 * width_court) - coordinate_y)) ** 2)
    else:
        distance_to_goal = math.sqrt((coordinate_x) ** 2 + (abs((0.5 * width_court) - coordinate_y)) ** 2)
    return distance_to_goal

# Xgoal : Helper to calculate angle to opponent's goal
def calculate_angle_to_goal(goalline_width, length_court, width_court, coordinate_x, coordinate_y, is_home_team):
    if is_home_team:
        coordinate_x_post_1, coordinate_x_post_2 = (length_court, length_court)
    else:
        coordinate_x_post_1, coordinate_x_post_2 = (0, 0)
    coordinate_y_post_1 = (width_court / 2) + (goalline_width / 2)
    coordinate_y_post_2 = (width_court / 2) - (goalline_width / 2)

    distance_to_post_1 = math.sqrt(abs(coordinate_x - coordinate_x_post_1) ** 2 + abs(coordinate_y - coordinate_y_post_1) ** 2)
    distance_to_post_2 = math.sqrt(abs(coordinate_x - coordinate_x_post_2) ** 2 + abs(coordinate_y - coordinate_y_post_2) ** 2)

    return math.acos((distance_to_post_1 ** 2 + distance_to_post_2 ** 2 - goalline_width ** 2) / (2 * distance_to_post_1 * distance_to_post_2))

In [10]:
# CLASS DATA STRUCTURE FOR STORING INFORMATION BASED ON INPUT
class Skill_Player_Values:
    def __init__(self, player_id):
        self.player_id = player_id
        self.attribute_values = {
            "height_cm" : 0,
            "weight_kgs" : 0,
            "age" : 0
        }
        self.skill_values = {
            "acceleration" : 0, "aggression" : 0, 
            "agility" : 0, "balance" : 0, 
            "ball_control" : 0,"composure" : 0, 
            "crossing" : 0, "curve" : 0, 
            "dribbling" : 0, "finishing" : 0,
            "freekick_accuracy" : 0, "heading_accuracy" : 0, 
            "interceptions" : 0, "jumping" : 0, 
            "long_passing" : 0, "long_shots" : 0, 
            "marking" : 0, "penalties" : 0, 
            "positioning" : 0, "reactions" : 0,
            "shot_power" : 0, "sliding_tackle" : 0, 
            "sprint_speed" : 0, "stamina" : 0, 
            "short_passing" : 0, "standing_tackle" : 0, 
            "strength" : 0, "vision" : 0, 
            "volleys" : 0
        }

    def get_player_id(self):
        return self.player_id
    
    def set_player_id(self, player_id):
        self.player_id = player_id

    def get_skill_value_by_key(self, key):
        return self.skill_values[key]
    
    def set_skill_value_by_key(self, key, score):
        self.skill_values[key] = score

    def get_attribute_value_by_key(self, key):
        return self.attribute_values[key]
    
    def set_attribute_value_by_key(self, key, score):
        self.attribute_values[key] = score

class Node_Decision_Making:
    def __init__(self, player_id, coordinate_x, coordinate_y, is_home_team):
        self.player_id = player_id
        self.coordinate = (coordinate_x, coordinate_y)
        self.xgoal_value = None
        self.is_home_team = is_home_team
        self.skill_and_attributes_values = Skill_Player_Values(player_id) 
    
    def get_player_id(self):
        return self.player_id
    
    def set_player_id(self, player_id):
        self.player_id = player_id
    
    def get_xgoal_value(self):
        return self.xgoal_value
    
    def set_xgoal_value(self, xgoal_value):
        self.xgoal_value = xgoal_value

    def get_coordinate_value(self):
        return self.coordinate
    
    def set_coordinate_value(self, coordinate_x, coordinate_y):
        self.coordinate = (coordinate_x, coordinate_y)

    def get_is_home_team(self):
        return self.is_home_team
    
    def set_is_home_team(self, is_home_team):
        self.is_home_team = is_home_team

    def get_skill_values(self):
        return self.skill_and_attributes_values
    
    def set_skill_values(self, object_skill_values):
        self.skill_and_attributes_values = object_skill_values

class Path_Decision_Making_Real:
    ACTION_TYPE_PASS = "pass"
    ACTION_TYPE_DRIBBLE = "dribble"
    ACTION_TYPE_SHOT = "shot"

    def __init__(self):
        self.route_path_per_timestamp_map = {}
        self.opponent_state_per_timestamp_map = {}

    def get_route_path_per_timestamp_map(self):
        return self.route_path_per_timestamp_map

    def get_opponent_state_per_timestamp_map(self):
        return self.opponent_state_per_timestamp_map

    def set_route_path_per_timestamp_element(self, timestamp, node_source, node_target):
        player_id_src = node_source.get_player_id()
        player_id_target = node_target.get_player_id()
        action_type = Path_Decision_Making_Real.ACTION_TYPE_PASS \
                        if player_id_src != player_id_target \
                        else Path_Decision_Making_Real.ACTION_TYPE_DRIBBLE
        self.route_path_per_timestamp_map[timestamp] = {
            "source" : node_source,
            "destination" : node_target,
            "action_type" : action_type
        }

    def set_opponent_state_per_timestamp_element(self, timestamp, opponent_node_list):
        self.opponent_state_per_timestamp_map[timestamp] = opponent_node_list

In [11]:
def construct_simple_maps_for_path_route_from_input(df_coordinate_player_teammates):
    unique_timestamp_teammates = sorted(df_coordinate_player_teammates['timestamp'].unique())
    simple_maps_path_route = {}
    stop_this_loop = False
    print("Please choose the desired corresponding routes.")
    for timestamp in unique_timestamp_teammates:
        df_these_players_timestamp = df_coordinate_player_teammates[df_coordinate_player_teammates['timestamp'] == timestamp]
        player_ids_this_timestamp = df_these_players_timestamp['player_id'].unique()
        request_input_stop = False
        while (not request_input_stop):
            system('cls')
            try:
                print(f'Choose one of these player(s) for timestamp {timestamp} - Type correct player id number')
                for _, df_row_player in df_these_players_timestamp.iterrows():
                    print("===================================================")
                    print(f'Player ID : {df_row_player["player_id"]} - Name : {df_row_player["name"]}')
                    print(f'Coordinate (x,y) in pitch : ({df_row_player["coordinate_x"]}, {df_row_player["coordinate_y"]})')
                input_player_id = input("\nEnter player id : ")
                if (input_player_id == "STOP"):
                    request_input_stop = True
                    stop_this_loop = True
                elif (int(input_player_id) in player_ids_this_timestamp):
                    simple_maps_path_route[timestamp] = int(input_player_id)
                    request_input_stop = True
                else:
                    print("Error : wrong input\n")
            except ValueError:
                print("Error : wrong input\n")
            finally:
                system('cls')
        if (stop_this_loop):
            break
    
    if (len(list(simple_maps_path_route.keys())) < 2):
        return {}
    simple_maps_path_route_final = {}
    for idx in range(len(unique_timestamp_teammates)-1):
        simple_maps_path_route_final[unique_timestamp_teammates[idx]] = {
            "source": simple_maps_path_route[unique_timestamp_teammates[idx]],
            "destination": simple_maps_path_route[unique_timestamp_teammates[idx+1]]
        }
    return simple_maps_path_route_final

def construct_path_decision_making_test_object(include_skill=False, include_opponent=False):
    df_coordinate_player_input = load_pandas_dataframe_from_csv(INPUT_RAW_PLAYER_COORDINATE_FILEPATH, delimiter=";")
    df_attribute_player_input = load_pandas_dataframe_from_csv(INPUT_PLAYER_ATTRIBUTE_FILE_CSV_PATH)
    df_skill_player_input = load_pandas_dataframe_from_csv(INPUT_PLAYER_SKILL_FILE_CSV_PATH)
    df_coordinate_player_teammates = df_coordinate_player_input[df_coordinate_player_input["is_opponent_team"] == 0].copy()
    df_coordinate_player_opponents = df_coordinate_player_input[df_coordinate_player_input["is_opponent_team"] == 1].copy()
    
    simple_maps_path_route = construct_simple_maps_for_path_route_from_input(df_coordinate_player_teammates)
    sorted_timestamp_list = sorted(list(simple_maps_path_route.keys()))
    path_decision_making = Path_Decision_Making_Real()
    for idx in range(len(sorted_timestamp_list)-1):
        player_id_source = simple_maps_path_route[sorted_timestamp_list[idx]]["source"]
        player_id_dest = simple_maps_path_route[sorted_timestamp_list[idx]]["destination"]
        source_x, source_y = df_coordinate_player_teammates.loc[(df_coordinate_player_teammates["player_id"] == player_id_source) & \
                                (df_coordinate_player_teammates["timestamp"] == sorted_timestamp_list[idx]), ["coordinate_x", "coordinate_y"]].iloc[0]
        dest_x, dest_y = df_coordinate_player_teammates.loc[(df_coordinate_player_teammates["player_id"] == player_id_dest) & \
                                (df_coordinate_player_teammates["timestamp"] == sorted_timestamp_list[idx+1]), ["coordinate_x", "coordinate_y"]].iloc[0]
        # Construct node player source
        node_player_source = Node_Decision_Making(player_id_source, source_x, source_y, IS_HOME_TEAM_DIRECTION)
        if (include_skill):
            df_attribute_player = df_attribute_player_input.loc[df_attribute_player_input["player_id"] == player_id_source, COLUMNS_PLAYERS_ATTRIBUTE]
            if (not df_attribute_player.empty):
                for attribute in COLUMNS_PLAYERS_ATTRIBUTE:
                    node_player_source.get_skill_values().set_attribute_value_by_key(attribute, df_attribute_player[attribute].iloc[0])
            df_skill_player = df_skill_player_input.loc[df_skill_player_input["player_id"] == player_id_source, COLUMNS_PLAYERS_SKILL]
            if (not df_skill_player.empty):
                for skill in COLUMNS_PLAYERS_SKILL:
                    node_player_source.get_skill_values().set_skill_value_by_key(skill, df_skill_player[skill].iloc[0])
        # Construct node player target
        node_player_target = Node_Decision_Making(player_id_dest, dest_x, dest_y, IS_HOME_TEAM_DIRECTION)
        if (include_skill):
            df_attribute_player = df_attribute_player_input.loc[df_attribute_player_input["player_id"] == player_id_dest, COLUMNS_PLAYERS_ATTRIBUTE]
            if (not df_attribute_player.empty):
                for attribute in COLUMNS_PLAYERS_ATTRIBUTE:
                    node_player_target.get_skill_values().set_attribute_value_by_key(attribute, df_attribute_player[attribute].iloc[0])
            df_skill_player = df_skill_player_input.loc[df_skill_player_input["player_id"] == player_id_dest, COLUMNS_PLAYERS_SKILL]
            if (not df_skill_player.empty):
                for skill in COLUMNS_PLAYERS_SKILL:
                    node_player_target.get_skill_values().set_skill_value_by_key(skill, df_skill_player[skill].iloc[0])
        # Append both node player source and target for this timestamp
        path_decision_making.set_route_path_per_timestamp_element(sorted_timestamp_list[idx], node_player_source, node_player_target)

    if (include_opponent):
        unique_player_ids_opponent = df_coordinate_player_opponents['player_id'].unique()
        for timestamp in sorted_timestamp_list:
            maps_player_id_with_timestamp = {}
            for player_id in unique_player_ids_opponent:
                df_opponent_player = df_coordinate_player_opponents[(df_coordinate_player_opponents['player_id'] == player_id) & (df_coordinate_player_opponents['timestamp'] == timestamp)]
                if (not df_opponent_player.empty):
                    maps_player_id_with_timestamp[player_id] = timestamp
                else:
                    df_opponent_player_without_timestamp = df_coordinate_player_opponents[(df_coordinate_player_opponents['player_id'] == player_id)].sort_values(by="timestamp")
                    if (not df_opponent_player_without_timestamp.empty):
                        maps_player_id_with_timestamp[player_id] = df_opponent_player_without_timestamp["timestamp"].iloc[0]
            list_opponent_objects_this_timestamp = []
            for player_id in sorted(list(maps_player_id_with_timestamp.keys())):
                player_timestamp_available = maps_player_id_with_timestamp[player_id]
                coordinate_x, coordinate_y = df_coordinate_player_opponents.loc[(df_coordinate_player_opponents["player_id"] == player_id) & \
                                (df_coordinate_player_opponents["timestamp"] == player_timestamp_available), ["coordinate_x", "coordinate_y"]].iloc[0]
                opponent_node_player = Node_Decision_Making(player_id, coordinate_x, coordinate_y, not IS_HOME_TEAM_DIRECTION)
                list_opponent_objects_this_timestamp.append(opponent_node_player)
            path_decision_making.set_opponent_state_per_timestamp_element(timestamp, list_opponent_objects_this_timestamp)

    return path_decision_making


In [12]:
INCLUDE_SKILL = True
INCLUDE_OPPONENT = True

path_decision_making_obj = construct_path_decision_making_test_object(include_skill=INCLUDE_SKILL, include_opponent=INCLUDE_OPPONENT)

Please choose the desired corresponding routes.
Choose one of these player(s) for timestamp 1 - Type correct player id number
Player ID : 1 - Name : Andi
Coordinate (x,y) in pitch : (28, 51)
Player ID : 2 - Name : Budi
Coordinate (x,y) in pitch : (36, 18)
Player ID : 3 - Name : Anto
Coordinate (x,y) in pitch : (43, 34)
Choose one of these player(s) for timestamp 2 - Type correct player id number
Player ID : 1 - Name : Andi
Coordinate (x,y) in pitch : (40, 51)
Player ID : 2 - Name : Budi
Coordinate (x,y) in pitch : (43, 18)
Player ID : 3 - Name : Anto
Coordinate (x,y) in pitch : (46, 34)
Choose one of these player(s) for timestamp 3 - Type correct player id number
Player ID : 1 - Name : Andi
Coordinate (x,y) in pitch : (52, 51)
Player ID : 2 - Name : Budi
Coordinate (x,y) in pitch : (50, 18)
Player ID : 3 - Name : Anto
Coordinate (x,y) in pitch : (49, 34)
Choose one of these player(s) for timestamp 4 - Type correct player id number
Player ID : 1 - Name : Andi
Coordinate (x,y) in pitch :